In [8]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px


# Cargar datos
file_path = "modelo_con_predicciones.csv"  # Asegúrate de tener el archivo en la ruta correcta
df = pd.read_csv(file_path)

# Crear categorías de precio
bins = [df['y_actual'].min(), df['y_actual'].quantile(0.33), df['y_actual'].quantile(0.66), df['y_actual'].max()]
labels = ['Bajo', 'Medio', 'Alto']
df['price_category'] = pd.cut(df['y_actual'], bins=bins, labels=labels)

# Inicializar la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout del dashboard
app.layout = dbc.Container([
    html.H1("Dashboard de Análisis de Alquileres"),
    
    dbc.Row([
        dbc.Col([
            html.Label("Seleccionar Ciudad:"),
            dcc.Dropdown(
                id='city-dropdown',
                options=[{'label': city, 'value': city} for city in df.filter(like='cityname_').columns],
                multi=True,
                placeholder="Seleccione ciudades"
            )
        ], width=6),
        dbc.Col([
            html.Label("Seleccionar Estado:"),
            dcc.Dropdown(
                id='state-dropdown',
                options=[{'label': state, 'value': state} for state in df.filter(like='state_').columns],
                multi=True,
                placeholder="Seleccione estados"
            )
        ], width=6)
    ]),
    
    html.Hr(),
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='scatter-price-sqft')
        ], width=6),
        dbc.Col([
            dcc.Graph(id='boxplot-price-city')
        ], width=6)
    ]),
    
    html.Hr(),
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='amenities-impact')
        ], width=6),
        dbc.Col([
            dcc.Graph(id='price-segmentation')
        ], width=6)
    ])
])

# Callbacks
@app.callback(
    [Output('scatter-price-sqft', 'figure'),
     Output('boxplot-price-city', 'figure'),
     Output('amenities-impact', 'figure'),
     Output('price-segmentation', 'figure')],
    [Input('city-dropdown', 'value'),
     Input('state-dropdown', 'value')]
)
def update_graphs(selected_cities, selected_states):
    filtered_df = df.copy()
    
    if selected_cities:
        city_cols = [col for col in df.columns if col in selected_cities]
        filtered_df = filtered_df[(filtered_df[city_cols] == 1).any(axis=1)]
    
    if selected_states:
        state_cols = [col for col in df.columns if col in selected_states]
        filtered_df = filtered_df[(filtered_df[state_cols] == 1).any(axis=1)]
    
    scatter_fig = px.scatter(filtered_df, x='square_feet', y='y_actual', color='price_category',
                             title="Precio vs. Metros Cuadrados", labels={'y_actual': 'Precio'})
    
    boxplot_fig = px.box(filtered_df, x='price_category', y='y_actual',
                          title="Distribución de precios por categoría")
    
    amenities_cols = [col for col in df.columns if col in ['Cable or Satellite', 'Pool', 'Parking']]
    amenities_impact = filtered_df[amenities_cols + ['y_actual']].melt(id_vars='y_actual')
    amenities_fig = px.box(amenities_impact, x='variable', y='y_actual', title="Impacto de Amenidades en Precio")
    
    segmentation_fig = px.histogram(filtered_df, x='price_category', title="Segmentación de Precios")
    
    return scatter_fig, boxplot_fig, amenities_fig, segmentation_fig

# Ejecutar el servidor
if __name__ == '__main__':
    app.run_server(debug=True)


In [11]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# Cargar los datos
file_path = 'predicciones_nn.csv'  # Ajusta la ruta según corresponda
df = pd.read_csv(file_path)

# Reconstrucción de cityname
df.columns = df.columns.str.strip()  # Eliminar espacios extra
df['cityname'] = df[[col for col in df.columns if col.startswith('cityname_')]].idxmax(axis=1).str.replace('cityname_', '', regex=True)

df['state'] = df[[col for col in df.columns if col.startswith('state_')]].idxmax(axis=1).str.replace('state_', '', regex=True)

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Diseño del tablero
app.layout = html.Div([
    html.H1("Tablero de Análisis de Apartamentos"),
    
    # Filtros
    html.Label("Selecciona una ciudad:"),
    dcc.Dropdown(
        id='city-dropdown',
        options=[{'label': city, 'value': city} for city in df['cityname'].unique()],
        multi=True,
        placeholder="Selecciona una o más ciudades"
    ),
    
    html.Label("Selecciona un estado:"),
    dcc.Dropdown(
        id='state-dropdown',
        options=[{'label': state, 'value': state} for state in df['state'].unique()],
        multi=True,
        placeholder="Selecciona uno o más estados"
    ),
    
    # Gráfico de dispersión
    dcc.Graph(id='scatter-plot'),

    # Mapa georreferenciado
    dcc.Graph(id='map-plot'),
])

# Callback para actualizar los gráficos
@app.callback(
    dash.dependencies.Output('scatter-plot', 'figure'),
    dash.dependencies.Output('map-plot', 'figure'),
    [dash.dependencies.Input('city-dropdown', 'value'),
     dash.dependencies.Input('state-dropdown', 'value')]
)
def update_graphs(selected_cities, selected_states):
    filtered_df = df.copy()
    
    if selected_cities:
        filtered_df = filtered_df[filtered_df['cityname'].isin(selected_cities)]
    if selected_states:
        filtered_df = filtered_df[filtered_df['state'].isin(selected_states)]
    
    scatter_fig = px.scatter(
        filtered_df, x='square_feet', y='y_predicted_nn',
        color='bathrooms_bedrooms_avg', title="Precio vs. Metros Cuadrados"
    )
    
    map_fig = px.scatter_mapbox(
        filtered_df, lat='latitude', lon='longitude', color='y_predicted_nn',
        size_max=15, zoom=10, mapbox_style="open-street-map",
        title="Mapa de Precios por Ubicación"
    )
    
    return scatter_fig, map_fig

if __name__ == '__main__':
    app.run_server(debug=True)



/var/folders/t2/cqb32kwx0bd46dt6hwft8kph0000gn/T/ipykernel_25245/2004404431.py:67: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [12]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# Cargar los datos
file_path = 'predicciones_rf.csv'  # Archivo de predicciones Random Forest
df = pd.read_csv(file_path)

# Reconstrucción de cityname y state
df.columns = df.columns.str.strip()
df['cityname'] = df[[col for col in df.columns if col.startswith('cityname_')]].idxmax(axis=1).str.replace('cityname_', '', regex=True)
df['state'] = df[[col for col in df.columns if col.startswith('state_')]].idxmax(axis=1).str.replace('state_', '', regex=True)

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Diseño del tablero
app.layout = html.Div([
    html.H1("Tablero de Análisis de Apartamentos - Random Forest"),
    
    # Filtros
    html.Label("Selecciona una ciudad:"),
    dcc.Dropdown(
        id='city-dropdown',
        options=[{'label': city, 'value': city} for city in df['cityname'].unique()],
        multi=True,
        placeholder="Selecciona una o más ciudades"
    ),
    
    html.Label("Selecciona un estado:"),
    dcc.Dropdown(
        id='state-dropdown',
        options=[{'label': state, 'value': state} for state in df['state'].unique()],
        multi=True,
        placeholder="Selecciona uno o más estados"
    ),
    
    # Gráfico de barras para comparar precios por ciudad
    dcc.Graph(id='bar-plot'),
    
    # Mapa georreferenciado
    dcc.Graph(id='map-plot'),
])

# Callback para actualizar los gráficos
@app.callback(
    dash.dependencies.Output('bar-plot', 'figure'),
    dash.dependencies.Output('map-plot', 'figure'),
    [dash.dependencies.Input('city-dropdown', 'value'),
     dash.dependencies.Input('state-dropdown', 'value')]
)
def update_graphs(selected_cities, selected_states):
    filtered_df = df.copy()
    
    if selected_cities:
        filtered_df = filtered_df[filtered_df['cityname'].isin(selected_cities)]
    if selected_states:
        filtered_df = filtered_df[filtered_df['state'].isin(selected_states)]
    
    bar_fig = px.bar(
        filtered_df.groupby('cityname')['Predicción_RF'].mean().reset_index(),
        x='cityname', y='Predicción_RF',
        title="Promedio de Predicción RF por Ciudad",
        labels={'Predicción_RF': 'Precio Predicho'}
    )
    
    map_fig = px.scatter_mapbox(
        filtered_df, lat='latitude', lon='longitude', color='Predicción_RF',
        size_max=15, zoom=10, mapbox_style="open-street-map",
        title="Mapa de Predicciones RF por Ubicación"
    )
    
    return bar_fig, map_fig

if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/t2/cqb32kwx0bd46dt6hwft8kph0000gn/T/ipykernel_25245/2386455266.py:68: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/t2/cqb32kwx0bd46dt6hwft8kph0000gn/T/ipykernel_25245/2386455266.py:68: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/t2/cqb32kwx0bd46dt6hwft8kph0000gn/T/ipykernel_25245/2386455266.py:68: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/t2/cqb32kwx0bd46dt6hwft8kph0000gn/T/ipykernel_25245/2386455266.py:68: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

